In [5]:
def train(in_alpha, in_l1_ratio):
    import logging
    import warnings
    import numpy as np
    import pandas as pd
    import sklearn
    from sklearn.linear_model import ElasticNet
    from sklearn.model_selection import train_test_split
    from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
    import mlflow 
    import mlflow.sklearn
    from mlflow.models import infer_signature
    logging.basicConfig(level=logging.WARN)
    logger=logging.getLogger(__name__)  #???

    def eval_metrics(actual,pred):
        rmse=np.sqrt(mean_squared_error(actual,pred))
        mae=mean_absolute_error(actual,pred)
        r2=r2_score(actual,pred)
        return rmse,mae,r2 
    
    warnings.filterwarnings('ignore')
    np.random.seed(34)
    
    # fetch data
    csv_url=('http://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv')
    try:
        data=pd.read_csv(csv_url, sep=';')
    except Exception as e:
        logger.exception('no file exists')
    
    # ML
    train, test=train_test_split(data)
    train_y,test_y=train[['quality']], test[['quality']]
    train_x,test_x=train.drop(['quality'],axis=1), test.drop(['quality'],axis=1)
    
    if float(in_alpha) is None:
        alpha=.5
    else:
        alpha=float(in_alpha)
    if float(in_l1_ratio) is None:
        l1_ratio=.5
    else:
        l1_ratio=float(in_l1_ratio)

    with mlflow.start_run():
        regression=ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=34)
        regression.fit(train_x, train_y)
        pre= regression.predict(test_x)
        (rmse,mae,r2)=eval_metrics(test_y,pre)

        print(f'elasticnet model(alpha={alpha:f}, l1={l1_ratio:f})')
        print('rmse:%s'%rmse)
        print('mae:%s'%mae)
        print('r2:%s'%r2)

        predictions=regression.predict(train_x)  
        signature=infer_signature(train_x, predictions)   #output is a dataframe like table; why training set?

        #mlflow
        mlflow.log_param('alpha', alpha)
        mlflow.log_param('l1_ratio', l1_ratio)
        mlflow.log_metric('rmse', rmse)
        mlflow.log_metric('mae',mae)
        mlflow.log_metric('r2',r2)

        mlflow.sklearn.log_model(regression,'model',signature=signature)




In [6]:
train(.5, .5)

elasticnet model(alpha=0.500000, l1=0.500000)
rmse:0.7746160676272058
mae:0.6189198753382797
r2:0.13161828976991063


In [23]:
train(.6,.3)

elasticnet model(alpha=0.600000, l1=0.300000)
rmse:0.7635105696953153
mae:0.6046901038382586
r2:0.15633938993962904


In [11]:
!cd C:\Users\Robin\learn-git\regression\mlruns

In [22]:
!mlflow ui

^C


In [17]:
! pip install Werkzeug==2.3.7

  Attempting uninstall: Werkzeug
    Found existing installation: werkzeug 3.0.0
    Uninstalling werkzeug-3.0.0:
      Successfully uninstalled werkzeug-3.0.0


In [15]:
! pip install Flask==2.2.2


  Using cached werkzeug-3.0.0-py3-none-any.whl (226 kB)
  Attempting uninstall: Werkzeug
    Found existing installation: Werkzeug 2.0.2
    Uninstalling Werkzeug-2.0.2:
      Successfully uninstalled Werkzeug-2.0.2
  Attempting uninstall: Flask
    Found existing installation: Flask 1.1.2
    Uninstalling Flask-1.1.2:
      Successfully uninstalled Flask-1.1.2


In [19]:
!mlflow ui --host 0.0.0.0

^C


In [7]:
import sys
sys.argv

['C:\\Users\\Robin\\AppData\\Roaming\\Python\\Python310\\site-packages\\ipykernel_launcher.py',
 '--ip=127.0.0.1',
 '--stdin=9011',
 '--control=9009',
 '--hb=9008',
 '--Session.signature_scheme="hmac-sha256"',
 '--Session.key=b"80680fdd-0eef-42fb-ae6e-7e7e60b47d90"',
 '--shell=9010',
 '--transport="tcp"',
 '--iopub=9012',
 '--f=c:\\Users\\Robin\\AppData\\Roaming\\jupyter\\runtime\\kernel-v2-15932OermygJSinQt.json']

In [24]:

print('tracking uri:', mlflow.get_tracking_uri())
print('artifact uri:', mlflow.get_artifact_uri())

NameError: name 'mlflow' is not defined